In [18]:
import pandas as pd
import pdb
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Lambda, Dense, Flatten, Reshape, merge, Highway, Activation,Dropout
from keras.layers.convolutional import Conv3D
from keras.layers.pooling import GlobalMaxPooling3D, MaxPooling3D, AveragePooling3D,GlobalAveragePooling3D
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianDropout
from keras.optimizers import Adamax, Adam, Nadam
from keras.layers.advanced_activations import ELU,PReLU,LeakyReLU
from keras import backend as K
from sklearn.preprocessing import LabelBinarizer
from keras.regularizers import l2
from pylab import imshow, show
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import os
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,LearningRateScheduler
from sklearn.metrics import accuracy_score, log_loss,roc_auc_score
from sklearn.linear_model import LogisticRegression
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import scipy.ndimage
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from skimage import measure, morphology
import SimpleITK as sitk
from PIL import Image
from scipy import ndimage
import threading
from multiprocessing import Process
from keras.regularizers import l2

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

%matplotlib inline

In [19]:
def conv_block(x_input, num_filters, pool=True, norm=True, drop_rate=0.0):

    x1 = Conv3D(num_filters,3,3,3,border_mode='same', W_regularizer=l2(1e-4), data_format='channels_first')(x_input)
    
    if norm:
        x1 = BatchNormalization(axis=1)(x1)
    
    x1 = GaussianDropout(drop_rate)(x1)
    x1 = LeakyReLU(.1)(x1)
    
    if pool:
        x1 = MaxPooling3D(dim_ordering="th")(x1)
        
    x_out = x1
    
    return x_out

In [20]:
def dense_branch(xstart, name, outSize=1, activation='sigmoid'):
    
    xdense = Dense(32, W_regularizer=l2(1e-4))(xstart)
    xdense = BatchNormalization()(xdense)
#     xdense = GaussianDropout(0.1)(xdense)
    xdense = LeakyReLU(.1)(xdense)
    xout = Dense(outSize, activation=activation, name=name, W_regularizer=l2(1e-3))(xdense)
    
    return xout

In [21]:
def build_model(input_shape, drop_rate):

    xin = Input(input_shape)

    x1 = conv_block(xin, 8, drop_rate=drop_rate) #outputs 9 ch
    x1_ident = AveragePooling3D(dim_ordering="th")(xin)
    x1_merged = merge([x1, x1_ident], mode='concat', concat_axis=1)
    
    x2_1 = conv_block(x1_merged, 24, drop_rate=drop_rate) #outputs 16+9 ch  = 25
    x2_ident = AveragePooling3D(dim_ordering="th")(x1_ident)
    x2_merged = merge([x2_1, x2_ident], mode='concat', concat_axis=1)
    
    #by branching we reduce the #params
    x3_1 = conv_block(x2_merged,64,norm=True,drop_rate=drop_rate) #outputs 25 + 16 ch = 41
    x3_ident = AveragePooling3D(dim_ordering="th")(x2_ident)
    x3_merged = merge([x3_1,x3_ident],mode='concat', concat_axis=1)

    x4_1 = conv_block(x3_merged,72,norm=True,drop_rate=drop_rate) #outputs 25 + 16 ch = 41
    x4_ident = AveragePooling3D(dim_ordering="th")(x3_ident)
    x4_merged = merge([x4_1,x4_ident],mode='concat', concat_axis=1)
    

    x5_1 = conv_block(x4_merged, 72, norm=True, pool=False, drop_rate=drop_rate) #outputs 25 + 16 ch = 41

    xpool = GlobalMaxPooling3D()(x5_1)
    xpool_norm = BatchNormalization()(xpool)

    xout_malig = dense_branch(xpool_norm,name='o_mal',outSize=1,activation='sigmoid')
    
    model = Model(input=xin, output=[xout_malig])

    
    if input_shape[1] == 32:
        lr_start = .005
    elif input_shape[1] == 64:
        lr_start = .001
    elif input_shape[1] == 128:
        lr_start = 1e-4
    elif input_shape[1] == 96:
        lr_start = 5e-4
    
    opt = Nadam(lr_start, clipvalue=1.0)
    print ('compiling model')

    model.compile(optimizer=opt,\
                  loss={'o_mal':'mean_squared_error'})
    return model

In [22]:
model_64 = build_model((1,64,64,64),0.0001)

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(8, (3, 3, 3), padding="same", data_format="channels_first", kernel_regularizer=<keras.reg...)`
  app.launch_new_instance()
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `AveragePooling3D` call to the Keras 2 API: `AveragePooling3D(data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/opt/anaconda/lib/python3.5/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Us

compiling model


/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, kernel_regularizer=<keras.reg...)`
  app.launch_new_instance()
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, name="o_mal", activation="sigmoid", kernel_regularizer=<keras.reg...)`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [23]:
def str_to_arr(arr_str,length):
    while '  ' in arr_str:
        arr_str = arr_str.replace('  ', ' ')
    result = eval(arr_str.replace('[ ', '[').replace(' ]', ']').replace(' ', ','))
    assert len(result) == length
    return np.array(result)

In [24]:
def batchGenerator(X1,X2,IX1,IX2,batch_size,augment=False,current_frac=.5):

    df = pd.read_csv('/home/katya/data/CSVFILES/annotations_enhanced.csv')
    
    #3 categories - [nodule, cad false positive, random subvoxel]

    df['malignancy'] /= 5.

    Ymalig = df['malignancy'].values

    while True:
        
        n1 = int(current_frac * batch_size)
        n2 = batch_size - n1
        
        ixs1 = np.random.choice(range(X1.shape[0]), size=n1, replace=False)
        ixs2 = np.random.choice(range(X2.shape[0]), size=n2, replace=False)
        Xbatch1, IXbatch1 = X1[ixs1], IX1[ixs1]
        Xbatch2, IXbatch2 = X2[ixs2], IX2[ixs2]
        
        Xbatch = np.concatenate([Xbatch1, Xbatch2], axis=0)
        IXbatch = np.concatenate([IXbatch1, IXbatch2], axis=0)
        IXbatch_eq_neg2 = (IXbatch == -2)
        IXbatch[IXbatch_eq_neg2] = -1
        #this way things that are -2 (cad false positives)
        #get labeled as non-nodules for everything else.
        #but we still know where they are.
        IXbatch_eq_neg1 = (IXbatch == -1)
        
        #making sure we don't use these.
        #row 32 is a big tumor so it should be obvious
        
        #normalize
        Xbatch = np.expand_dims(Xbatch, 1)
        if augment:
            Xbatch = random_perturb(Xbatch)
        Xbatch = Xbatch.astype('float32')
        Xbatch = (Xbatch + 1000.) / (400. + 1000.)
        Xbatch = np.clip(Xbatch,0,1)
        
        Ybatch_malig = Ymalig[IXbatch]
        Ybatch_malig[IXbatch_eq_neg1] = 0.0
        
        yield Xbatch, {'o_mal':Ybatch_malig}        

In [25]:
def stage_1_lr_schedule(i):
    if i < 10:
        return np.float32(.004)
    if i < 15:
        return np.float32(.002)
    if i < 20:
        return np.float32(.0005)
    if i < 23:
        return np.float32(.0001)
    return np.float32(3e-5)

def stage_2_lr_schedule(i):
    if i == 0:
        return np.float32(.001)
    if i < 5:
        return np.float32(.0004)
    if i < 10:
        return np.float32(.0002)
    return np.float32(3e-5)

In [31]:
def train_model_on_stage(model, split=True, test_size=0.2, batch_size=10):
    
    if split:
        
        Xpos = np.load('/home/katya/data/voxels_true64/subset0Xtrue.npy')
        IXpos = np.load('/home/katya/data/voxels_true64/subset0IXtrue.npy')
        
        for subset in range(1,10):
            XposTemp = np.load('/home/katya/data/voxels_true64/subset' + str(subset) + 'Xtrue.npy')
            IXposTemp = np.load('/home/katya/data/voxels_true64/subset' + str(subset) + 'IXtrue.npy')
            
            Xpos = np.concatenate((Xpos, XposTemp),axis=0)
            IXpos = np.concatenate((IXpos, IXposTemp),axis=0)
            
        del XposTemp, IXposTemp
        
        Xtrain1,Xvalid1,IXtrain1,IXvalid1 = train_test_split(Xpos, IXpos, test_size=test_size)
        
        del Xpos, IXpos
        
        Xneg = np.load('/home/katya/data/voxels_random64/subset0Xrandom.npy')
        IXneg = np.load('/home/katya/data/voxels_random64/subset0IXrandom.npy')
        
        for subset in range(1,10):
            XnegTemp = np.load('/home/katya/data/voxels_true64/subset' + str(subset) + 'Xtrue.npy')
            IXnegTemp = np.load('/home/katya/data/voxels_true64/subset' + str(subset) + 'IXtrue.npy')
            
            Xneg = np.concatenate((Xneg, XnegTemp),axis=0)
            IXneg = np.concatenate((IXneg, IXnegTemp),axis=0)
            
        del XnegTemp, IXnegTemp
        
        Xtrain2,Xvalid2,IXtrain2,IXvalid2 = train_test_split(Xneg, IXneg, test_size=test_size)
        
        del Xneg, IXneg
        
        train_generator_75 = batchGenerator(Xtrain1,Xtrain2,IXtrain1,IXtrain2,\
                                                  augment=False, batch_size=batch_size,\
                                                  current_frac=.5)
        valid_generator_75 = batchGenerator(Xvalid1,Xvalid2,IXvalid1,IXvalid2,\
                                                  augment=False, batch_size=batch_size,\
                                                  current_frac=.5)
    
    else:

    
        subset=str(subset)
        Xpos = np.load('/home/katya/data/voxels_true64/subset' + subset + 'Xtrue.npy')
        IXpos = np.load('/home/katya/data/voxels_true64/subset' + subset + 'IXtrue.npy')
        Xneg = np.load('/home/katya/data/voxels_random64/subset' + subset + 'Xrandom.npy')
        IXneg = np.load('/home/katya/data/voxels_random64/subset' + subset + 'IXrandom.npy')
        train_generator_75 = batchGenerator(Xpos, Xneg, IXpos, IXneg, \
                                                  augment=False, batch_size=batch_size, \
                                                  current_frac=.5)


    name = '../model_and_weights/model_LUNA_v1_{epoch:02d}.h5'
    chkp = ModelCheckpoint(filepath=name)

    lr_schedule = LearningRateScheduler(stage_2_lr_schedule)
    nb_epoch = 25
    steps_per_epoch = 300
        
    lossHist = {}
    
    for lossType in ['loss','val_loss']:
        lossHist[lossType] = []
    
    for epoch in range(nb_epoch):
        hist = model.fit_generator(train_generator_75,validation_data=valid_generator_75, validation_steps=1,\
                                   steps_per_epoch=steps_per_epoch, \
                            nb_epoch=epoch+1,callbacks=[chkp],initial_epoch=epoch)
        for lossType in ['loss','val_loss']:
            lossHist[lossType].extend(hist.history[lossType])

    return model, lossHist

In [ ]:
# model_64.load_weights('/home/katya/LungCancer/Katya/CNN_v1/model_v1_weights_temp64.h5')
model_64, lossHist = train_model_on_stage(model_64, split=True, batch_size=30)
model_64.save_weights('/home/katya/LungCancer/Katya/CNN_v2/model_and_weights/model_v1_weights_temp64.h5')

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., initial_epoch=0, epochs=1, validation_data=<generator..., steps_per_epoch=300, validation_steps=1, callbacks=[<keras.ca...)`


Epoch 1/1
300/300 [==============================] - 1072s - loss: 0.0474 - val_loss: 0.0687

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:64: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., initial_epoch=1, epochs=2, validation_data=<generator..., steps_per_epoch=300, validation_steps=1, callbacks=[<keras.ca...)`


159/300 [==============>...............] - ETA: 505s - loss: 0.0453 

In [ ]:
plt.figure(figsize=[10,7])
plt.grid(True, ls='--', lw=0.5, alpha=0.5, dash_capstyle = 'round', c='gray')
plt.xlabel('Epoch #', fontsize=15)
plt.ylabel('Loss', fontsize=15)

colors = ['#F03861','#0D3446']

labels=list(lossHist.keys())

for i,key in enumerate(lossHist):
    plt.plot([x for x in range(15)], lossHist[key], '-', color=colors[i], label=labels[i])
    plt.legend()
    
plt.savefig('/home/katya/LungCancer/Katya/CNN_v2/figures/3.png')